This notebook is for experimenting with a CNN to PCA to RNN LSTM network model. The CNN will first have its dimensions reduced by PCA, then the LSTM network train on one CT scan at a time; it will take in the principal components of each slice in a scan and output predictions for ICH on each slice, while taking into account spacial dependencies.

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import os
import math
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
train_img_dir = '/home/jupyter/rsna-intracranial-hemorrhage-detection/stage_2_train_imgs/'
train_label_path = '/home/jupyter/rsna-intracranial-hemorrhage-detection/train_labels.csv'
train_ct_path = '/home/jupyter/rsna-intracranial-hemorrhage-detection/train_ct_scans.csv'
train_coord_path = '/home/jupyter/rsna-intracranial-hemorrhage-detection/train_ct_coords.csv'

test_img_dir = '/home/jupyter/rsna-intracranial-hemorrhage-detection/stage_2_test_imgs/'

keras.mixed_precision.set_global_policy('mixed_float16')
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: A100-SXM4-40GB, compute capability 8.0


In [5]:
base_model = keras.models.load_model('/home/jupyter/reproduce_training_2/checkpoint.ckpt')
extractor = keras.models.Sequential(base_model.layers[:-1])
extractor.summary()

2021-12-23 06:36:55.159535: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-23 06:36:55.693965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38444 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 16, 16, 2048)      42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 42,658,176
Trainable params: 42,552,832
Non-trainable params: 105,344
_________________________________________________________________


In [6]:
def get_img_tensor(img_path):
    return tf.convert_to_tensor(np.asarray(Image.open(img_path), dtype=np.float32) / 255.)


class RSNASequence(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = [self.y[img_id.split('/')[-1].split('.')[0]] for img_id in batch_x]
        
        return (tf.stack([get_img_tensor(img_path) for img_path in batch_x], axis=0), 
               tf.convert_to_tensor(batch_y))
    
    def on_epoch_end(self):
        ind = np.random.choice(list(range(len(os.listdir(train_img_dir)))), size=train_cutoff, replace=False)
        self.x = [train_img_dir + img_name for img_name in np.array(os.listdir(train_img_dir))[ind]]
    

In [7]:
labels = pd.read_csv(train_label_path)
labels = {l[0]: l[1:].astype(np.int8) for l in labels.to_numpy()}

In [8]:
batch_size = 32
train_cutoff = batch_size * 4000 #training the whole dataset takes ~9 hours, so we cut it short for proof-of-concept purposes.
ind = np.random.choice(list(range(len(os.listdir(train_img_dir)))), size=train_cutoff, replace=False)
train_sequence = RSNASequence([train_img_dir + img_name for img_name in np.array(os.listdir(train_img_dir))[ind]], labels, batch_size)

cp_callback = keras.callbacks.ModelCheckpoint(filepath='/home/jupyter/reproduce_training_2/checkpoint.ckpt',
                                                 save_weights_only=False,
                                                 verbose=1)

In [9]:
base_model.fit(x=train_sequence, epochs=3, callbacks=[cp_callback])

Epoch 1/3


2021-12-23 06:37:56.076647: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200


4000/4000 [==============================] - ETA: 0s - loss: 0.0751 - binary_accuracy: 0.9733 - auc: 0.9613 - precision: 0.8410 - recall: 0.6575
Epoch 00001: saving model to /home/jupyter/reproduce_training_2/checkpoint.ckpt


2021-12-23 07:30:29.796167: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/jupyter/reproduce_training_2/checkpoint.ckpt/assets


...

4000/4000 [==============================] - 3213s 797ms/step - loss: 0.0751 - binary_accuracy: 0.9733 - auc: 0.9613 - precision: 0.8410 - recall: 0.6575
Epoch 2/3
4000/4000 [==============================] - ETA: 0s - loss: 0.0708 - binary_accuracy: 0.9749 - auc: 0.9646 - precision: 0.8475 - recall: 0.6763
Epoch 00002: saving model to /home/jupyter/reproduce_training_2/checkpoint.ckpt
INFO:tensorflow:Assets written to: /home/jupyter/reproduce_training_2/checkpoint.ckpt/assets


/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


4000/4000 [==============================] - 2944s 736ms/step - loss: 0.0708 - binary_accuracy: 0.9749 - auc: 0.9646 - precision: 0.8475 - recall: 0.6763
Epoch 3/3
4000/4000 [==============================] - ETA: 0s - loss: 0.0676 - binary_accuracy: 0.9759 - auc: 0.9684 - precision: 0.8521 - recall: 0.6936
Epoch 00003: saving model to /home/jupyter/reproduce_training_2/checkpoint.ckpt
INFO:tensorflow:Assets written to: /home/jupyter/reproduce_training_2/checkpoint.ckpt/assets


/opt/conda/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.7/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


4000/4000 [==============================] - 2754s 689ms/step - loss: 0.0676 - binary_accuracy: 0.9759 - auc: 0.9684 - precision: 0.8521 - recall: 0.6936
